In [247]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string

In [249]:
df = pd.DataFrame()
site = 'https://www.hockey-reference.com'
for letter in string.ascii_lowercase:
    count = 0
    url = requests.get(site+'/players/'+letter+'/').text
    soup = BeautifulSoup(url,'lxml')
    player_list = soup.find_all('p', class_='nhl')

    for count, player in enumerate(player_list):
        if int(player.text.split('(')[-1][0:4])<1980 or player.text[-2] == 'G':
            continue
        if player.a.text == prev_player:
            if same_name_flag == True:
                same_name_count += 1
            same_name_flag = True
        else:
            same_name_flag = False
            same_name_count = 0
        player_name = player.a.text+same_name_flag*str(same_name_count)
        prev_player = player.a.text
        print(f'\r{count:03d}/{len(player_list)} of the letter {letter.upper()} completed, currently on: {player_name}                           ',end='')
        player_url = site+player.a['href']
        player_url = requests.get(player_url).text
        player_page = BeautifulSoup(player_url,'lxml')   
        player_year_log = player_page.find('li',class_ = 'full hasmore')
        for year in player_year_log.find_all('li'):
            if not year.text == 'Career Playoffs':
                game_log_url = site + year.a['href']
                game_log_url = requests.get(game_log_url).text

                try:
                    df_list = pd.read_html(game_log_url, skiprows = 1, header = 0)
                except:
                    continue
                tmp_df = df_list[0]
                tmp_df.rename(columns = {'Unnamed: 5':'Location','Unnamed: 7':'Result', 'G.1':'Goals','A':'Assists'}, inplace = True)
                tmp_df.drop(columns = ['+/-', 'EV', 'PP', 'SH', 'GW', 'EV.1', 'PP.1','SH.1', 'S', 'S%', 'SHFT', 'TOI',
                                   'HIT', 'BLK', 'FOW', 'FOL', 'FO%'], inplace=True)
                tmp_df.drop(tmp_df.loc[tmp_df['Rk']=='Rk'].index, inplace=True)
                tmp_df['Location'] = tmp_df.Location.apply(lambda x: 'A' if x == '@' else 'H')
                tmp_df['Age'] = tmp_df.Age.apply(lambda x: x[0:2])
                tmp_df['Player'] = player_name
                tmp_df[['G', 'Age', 'Goals', 'Assists', 'PTS', 'PIM']] = tmp_df[['G', 'Age', 'Goals', 'Assists', 'PTS', 'PIM']].apply(pd.to_numeric)
                if df.empty:
                    df = tmp_df
                else:
                    df = df.append(tmp_df, ignore_index=True)

059/60 of the letter Z completed, currently on: Andrei Zyuzin                                            

In [189]:
df = pd.DataFrame()
player_year_log = player_page.find('li',class_ = 'full hasmore')

for year in player_year_log.find_all('li'):
    if not year.text == 'Career Playoffs':
        game_log_url = site + year.a['href']
        game_log_url = requests.get(game_log_url).text
        
        df_list = pd.read_html(game_log_url, skiprows = 1, header = 0)
        tmp_df = df_list[-1]
        tmp_df.rename(columns = {'Unnamed: 5':'Location','Unnamed: 7':'Result', 'G.1':'Goals','A':'Assists'}, inplace = True)
        tmp_df.drop(columns = ['+/-', 'EV', 'PP', 'SH', 'GW', 'EV.1', 'PP.1','SH.1', 'S', 'S%', 'SHFT', 'TOI',
                           'HIT', 'BLK', 'FOW', 'FOL', 'FO%'], inplace=True)
        tmp_df.drop(tmp_df.loc[tmp_df['Rk']=='Rk'].index, inplace=True)
        tmp_df.drop(columns = ['Rk'], inplace = True)
        tmp_df['Location'] = tmp_df.Location.apply(lambda x: 'A' if x == '@' else 'H')
        tmp_df['Age'] = tmp_df.Age.apply(lambda x: x[0:2])
        tmp_df['Player'] = player_name
        tmp_df[['G', 'Age', 'Goals', 'Assists', 'PTS', 'PIM']] = tmp_df[['G', 'Age', 'Goals', 'Assists', 'PTS', 'PIM']].apply(pd.to_numeric)
        if df.empty:
            df = tmp_df
        else:
            df = df.append(tmp_df, ignore_index=True)
        
#print(player_year_log.find_all('li'))

In [159]:
df_list = pd.read_html(game_log_url, skiprows = 1, header = 0)
tmp_df = df_list[-1]
tmp_df.rename(columns = {'Unnamed: 5':'Location','Unnamed: 7':'Result', 'G.1':'Goals','A':'Assists'}, inplace = True)
tmp_df.drop(columns = ['+/-', 'EV', 'PP', 'SH', 'GW', 'EV.1', 'PP.1','SH.1', 'S', 'S%', 'SHFT', 'TOI',
                   'HIT', 'BLK', 'FOW', 'FOL', 'FO%'], inplace=True)
tmp_df.drop(tmp_df.loc[tmp_df['Rk']=='Rk'].index, inplace=True)
tmp_df['Location'] = tmp_df.Location.apply(lambda x: 'A' if x == '@' else 'H')
tmp_df['Age'] = tmp_df.Age.apply(lambda x: x[0:2])
tmp_df['Player'] = player_name

In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395913 entries, 0 to 1395912
Data columns (total 13 columns):
Rk          1395913 non-null object
Date        1395913 non-null object
G           1395913 non-null int64
Age         1395913 non-null int64
Tm          1395913 non-null object
Location    1395913 non-null object
Opp         1395913 non-null object
Result      1395913 non-null object
Goals       1395913 non-null int64
Assists     1395913 non-null int64
PTS         1395913 non-null int64
PIM         1395913 non-null int64
Player      1395913 non-null object
dtypes: int64(6), object(7)
memory usage: 138.4+ MB


In [262]:
df.describe()


,G,Age,Goals,Assists,PTS,PIM
count,1.395913e+06,1.395913e+06,1.395913e+06,1.395913e+06,1.395913e+06,1.395913e+06
mean,3.288734e+01,2.659149e+01,1.678070e-01,2.811916e-01,4.489986e-01,8.489555e-01
std,2.152099e+01,4.346565e+00,4.295467e-01,5.579892e-01,7.287368e-01,2.168806e+00
min,1.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.400000e+01,2.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.000000e+01,2.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e+01,2.900000e+01,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00
max,8.600000e+01,4.800000e+01,5.000000e+00,7.000000e+00,8.000000e+00,5.700000e+01


In [255]:
df.loc[df.PTS==8]

,Rk,Date,G,Age,Tm,Location,Opp,Result,Goals,Assists,PTS,PIM,Player
211088,70,1986-03-14,70,24,EDM,H,DET,W,2,6,8,0,Paul Coffey
389478,44,2012-02-02,44,22,EDM,H,CHI,W,4,4,8,0,Sam Gagner
444774,21,1983-11-19,21,22,EDM,H,NJD,W,3,5,8,2,Wayne Gretzky
444794,41,1984-01-04,41,22,EDM,H,MNS,W,4,4,8,0,Wayne Gretzky
712341,4,1988-10-15,4,23,PIT,H,STL,W,2,6,8,2,Mario Lemieux
712373,36,1988-12-31,36,23,PIT,H,NJD,W,5,3,8,2,Mario Lemieux
893580,26,1988-12-01,26,27,LAK,H,TOR,W,2,6,8,0,Bernie Nicholls
1194189,62,1981-02-22,62,21,QUE,A,WSH,W,3,5,8,0,Anton Stastny
1196103,59,1981-02-22,59,24,QUE,A,WSH,W,4,4,8,0,Peter Stastny


In [251]:
player_name

'Andrei Zyuzin'

In [252]:
year

<li><a href="/players/z/zyuzian01/gamelog/playoffs">Career Playoffs</a></li>

In [216]:
game_log_url2 = requests.get('https://www.hockey-reference.com/players/g/gavrivl01/gamelog/2019').text
try:
    df_list_2 = pd.read_html(game_log_url2, skiprows=1, header=0)
except:
    print('blah')


blah


In [214]:
print(df_list_2[0])

    Rk        Date   G     Age   Tm Unnamed: 5  Opp Unnamed: 7 G.1  A  ...  \
0    1  2002-10-11   1  20-239  MIN        NaN  BOS          W   0  0  ...   
1    2  2002-10-12   2  20-240  MIN          @  STL          T   0  2  ...   
2    3  2002-10-15   3  20-243  MIN        NaN  FLA          W   0  1  ...   
3    4  2002-10-17   4  20-245  MIN        NaN  DAL          W   2  0  ...   
4    5  2002-10-19   5  20-247  MIN        NaN  DET          L   0  0  ...   
..  ..         ...  ..     ...  ...        ...  ...        ...  .. ..  ...   
80  78  2003-03-31  78  21-045  MIN        NaN  CGY          W   0  1  ...   
81  79  2003-04-02  79  21-047  MIN          @  CBJ          L   0  0  ...   
82  80  2003-04-03  80  21-048  MIN          @  TOR          L   0  0  ...   
83  Rk        Date   G     Age   Tm        NaN  Opp        NaN   G  A  ...   
84  81  2003-04-06  81  21-051  MIN        NaN  CBJ          W   1  1  ...   

   SH.1  S    S%  SHFT    TOI  HIT  BLK  FOW  FOL  FO%  
0   Na

In [250]:
df.to_csv('NHL_game_log_all.csv',index=False)

In [241]:
prev_player = ''
same_name_count = 0
same_name_flag = False
for index,player in enumerate(player_list):
    if player.a.text == prev_player:
        if same_name_flag == True:
            same_name_count += 1
        same_name_flag = True
    else:
        same_name_flag = False
        same_name_count = 0
    player_name = player.a.text+same_name_flag*str(same_name_count)
    prev_player = player.a.text
    print(player_name)
        

AttributeError: 'str' object has no attribute 'a'

In [242]:
player_list = soup.find_all('p', class_='nhl')
print(player_list)

[<p class="nhl"><a href="/players/g/gaborma01.html">Marian Gaborik</a> (2001-2018, RW)</p>, <p class="nhl"><a href="/players/g/gabriku01.html">Kurtis Gabriel</a> (2016-2019, RW)</p>, <p class="nhl"><a href="/players/g/gadsbbi01.html">Bill Gadsby</a> (1947-1966, D)</p>, <p class="nhl"><a href="/players/g/gaetzli01.html">Link Gaetz</a> (1989-1992, D)</p>, <p class="nhl"><a href="/players/g/gagejo02.html">Joaquin Gage</a> (1995-2001, G)</p>, <p class="nhl"><a href="/players/g/gagejo01.html">Jody Gage</a> (1981-1992, RW)</p>, <p class="nhl"><a href="/players/g/gagnear01.html">Art Gagne</a> (1927-1932, RW)</p>, <p class="nhl"><a href="/players/g/gagnepa01.html">Paul Gagne</a> (1981-1990, LW)</p>, <p class="nhl"><a href="/players/g/gagnepi01.html">Pierre Gagne</a> (1960-1960, LW)</p>, <p class="nhl"><a href="/players/g/gagnesi01.html">Simon Gagne</a> (2000-2015, LW)</p>, <p class="nhl"><a href="/players/g/gagneda01.html">Dave Gagner</a> (1985-1999, C)</p>, <p class="nhl"><strong><a href="/pl

In [246]:
site = 'https://www.hockey-reference.com'
for letter in string.ascii_lowercase:
    count = 0
    url = requests.get(site+'/players/'+letter+'/').text
    soup = BeautifulSoup(url,'lxml')
    player_list = soup.find_all('p', class_='nhl')
    for player in player_list:
        print(player.a.text)
        if count >5:
            break
        count +=1
        
        
    

Antti Aalto
George Abbott
Reg Abbott
Spencer Abbott
Justin Abdelkader
Clarence Abel
Gerry Abel
Pete Babando
Anton Babchuk
Bobby Babcock
Warren Babe
Yuri Babenko
Mitch Babin
John Baby
Nicholas Caamano
Jack Caffery
Terry Caffery
Drake Caggiula
Larry Cahan
Charles Cahill
Francis Cain
Matt D'Agostini
Corrie D'Alessio
Jerry D'Amigo
Hank D'Amore
Marc D'Amour
Stephane Da Costa
Joey Daccord
Ben Eager
Mike Eagles
Bruce Eakin
Cody Eakin
Dallas Eakins
Robbie Earl
Mike Eastwood
Robby Fabbri
Dante Fabbro
Brian Fahey
Jim Fahey
Trevor Fahey
Bill Fairbairn
Cade Fairchild
Marian Gaborik
Kurtis Gabriel
Bill Gadsby
Link Gaetz
Joaquin Gage
Jody Gage
Art Gagne
Kari Haakana
Ari Haanpaa
Henrik Haapala
David Haas
Gaetan Haas
Marc Habscheid
Len Hachborn
Alex Iafallo
Al Iafrate
Mike Iggulden
Jarome Iginla
Victor Ignatjev
Miroslav Ihnacak
Peter Ihnacak
Pat Jablonski
Greg Jacina
Barret Jackman
Ric Jackman
Tim Jackman
Art Jackson
Busher Jackson
Jari Kaarela
Bob Kabel
Frantisek Kaberle
Tomas Kaberle
Mark Kachowski


In [263]:
df.columns

Index(['Rk', 'Date', 'G', 'Age', 'Tm', 'Location', 'Opp', 'Result', 'Goals',
       'Assists', 'PTS', 'PIM', 'Player'],
      dtype='object')

In [264]:
df.Tm.unique()

array(['MDA', 'TOR', 'CHI', 'DET', 'NSH', 'EDM', 'ANA', 'MIN', 'PHX',
       'PIT', 'ATL', 'HAR', 'OTT', 'BOS', 'FLA', 'MTL', 'MNS', 'PHI',
       'WSH', 'NYI', 'BUF', 'CBJ', 'CAR', 'VAN', 'QUE', 'NJD', 'DAL',
       'TBL', 'CGY', 'STL', 'COL', 'LAK', 'SJS', 'WPG', 'NYR', 'WIN',
       'CBH', 'ARI', 'CLR', 'ATF', 'VEG'], dtype=object)

In [265]:
df.Tm.nunique()

41

In [270]:
df.loc[df.Tm == 'CBH']

,Rk,Date,G,Age,Tm,Location,Opp,Result,Goals,Assists,PTS,PIM,Player
20673,1,1983-11-06,1,20,CBH,A,NJD,L,0,0,0,0,Darrel Anholt
73360,1,1984-10-21,1,19,CBH,H,LAK,W,0,2,2,0,Marc Bergevin
73361,2,1984-10-24,2,19,CBH,H,VAN,W,0,0,0,0,Marc Bergevin
73362,3,1984-10-27,3,19,CBH,A,STL,W,0,1,1,0,Marc Bergevin
73363,4,1984-10-28,4,19,CBH,H,HAR,L,0,0,0,0,Marc Bergevin
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373758,4,1981-01-24,4,22,CBH,A,STL,L,0,0,0,0,Brian Young
1373759,5,1981-02-25,5,22,CBH,H,WSH,T,0,0,0,0,Brian Young
1373760,6,1981-02-27,6,22,CBH,A,BUF,L,0,1,1,2,Brian Young
1373761,7,1981-03-01,7,22,CBH,H,STL,T,0,0,0,2,Brian Young
